In [28]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import import_ipynb #requires pip install

Import new dataset

In [29]:
users = pd.read_csv(r"D:\dataset\encoding\user_frame.csv")
anime = pd.read_csv(r"D:\dataset\encoding\anime_frame.csv")
ratings = pd.read_csv(r"D:\dataset\encoding\ratings_frame.csv")

#Function to turn "genres" columns from string into list
def string_to_list(genres):
    return eval(genres)

users["fav_genres"] = users["fav_genres"].apply(string_to_list)
anime["genre"] = anime["genre"].apply(string_to_list)
ratings["genre"] = ratings["genre"].apply(string_to_list)

In [30]:
#Build genre list
def split(genres):
    return str(genres).split(",")

def build_genre_list(dataframe):
    #Updating the set to get the individual genre names in the set instead of the original list of genres
    genre_set = set()
    for i in dataframe: 
        genre_set.update(i)
    result = list(genre_set)

    #Removing blank spaces
    final_result = []
    for i in result:
        if i!="nan":
            new = i.replace(" ", "")
            final_result.append(new)

    #Removing duplicates now that spaces have been removed
    genre_list = list(set(final_result))
    return genre_list

In [31]:


user_cols = users.loc[:, ["username", "experience","gender", "generation",  "fav_anime_period"]]

encoder = LabelEncoder()
#Function to create a separate feature for each column and vectorize
def genre_columns():

    user_genres = users.loc[:, ["username", "fav_genres"]]
    genre_list = build_genre_list(user_genres["fav_genres"])
 
    for i in genre_list:
        user_cols[i] = 0

    for x, y in user_genres.iterrows():
        user_row = user_cols.loc[user_cols["username"] == y["username"]]
        #print(y)
        #encode each of the three favourite genres with value: 1
        user_row[y["fav_genres"][0]] = 1
        user_row[y["fav_genres"][1]] = 1
        user_row[y["fav_genres"][2]] = 1
        user_cols.loc[user_cols["username"] == y["username"]] = user_row
    #for genres in user_genres:
        #user_cols[genres[0]] = 1
        #ser_cols[genres[2]] = 1
        #print(genres)
        #bag_vector = tfidf.transform(genres)
        
        #print(bag_vector.toarray())
    genre_cols = user_cols.drop(columns=["experience","gender", "generation","fav_anime_period"])
    return genre_cols

#Vectorise features of a single user
def vectorize_single(user):
    #Get row of one user
    user_row = user_cols.loc[user_cols["username"] == user]
    user_vector = user_row.copy()
    
    #Fit the encoder with the data of original df: user_cols
    for col in user_cols.columns:
        enc = encoder.fit(user_cols[col])
        #print(list(enc.classes_))
        user_vector[col]=enc.transform(user_vector[col]) #encode each column of target user

    return user_vector

#Vectorise features of all users in the dataframe
def vectorize_all():
    all_users = user_cols.copy()

    for user in user_cols["username"]:
        user_vector = vectorize_single(user)
        all_users.loc[all_users["username"] == user] = user_vector
    #Make sure username field is in string form for identifier    
    all_users["username"] = user_cols["username"]
    #K-Nearest Neighbours
    #nb = KNeighborsClassifier(n_neighbors=5)
    return all_users

#Construct final vector of encoded features
def construct_vectors():
    string_features = vectorize_all()
    genre_features = genre_columns()
    final_vector = pd.merge(string_features, genre_features, on = "username")
    return final_vector

construct_vectors().to_csv(r"D:\dataset\encoding\user_vectors.csv", index=False)
#genre_columns()
#vectorize_all()
#user_cols
# need to translate genres that user is asked to anime genres


C:\Users\titot\AppData\Local\Temp\ipykernel_664\3245093844.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_row[y["fav_genres"][0]] = 1
C:\Users\titot\AppData\Local\Temp\ipykernel_664\3245093844.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_row[y["fav_genres"][1]] = 1
C:\Users\titot\AppData\Local\Temp\ipykernel_664\3245093844.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [32]:
vectorize_single("badking95")


,username,experience,gender,generation,fav_anime_period,Slice,Dementia,Arts,Adventure,Vampire,...,Life,Drama,of,Ecchi,Police,Historical,Super,Magic,Thriller,Seinen
9301,6443,3,1,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [33]:

anime_vector = anime.loc[:, ["airing","studio", "genre", "time_period", "fame"]]
anime_vector
#anime
#anime_genres = users.loc[:, "genre"]
#genre_list = build_genre_list(anime_genres)
#for i in genre_list:
            #user_cols[i] = 0

,airing,studio,genre,time_period,fame
0,False,David Production,"[Comedy, Supernatural, Romance, Shounen]",New-Gen,famous
1,False,Gonzo,"[Comedy, Parody, Romance, School, Shounen]",Classic,semi-famous
2,False,Satelight,"[Comedy, Magic, School, Shoujo]",Classic,well-known
3,False,Hal Film Maker,"[Comedy, Drama, Magic, Romance, Fantasy]",Classic,semi-famous
4,False,J.C.Staff,"[Comedy, Drama, Romance, Shounen]",New-Gen,semi-famous
...,...,...,...,...,...
3235,False,Millepensee,"[Music, Drama]",New-Gen,niche
3236,False,Production I.G,"[Comedy, Sports, School, Shounen]",New-Gen,famous
3237,False,"Satelight, Production Reed","[Action, Mecha, Romance, Super Power, Supe...",Classic,well-known
3238,False,Toei Animation,"[Adventure, Fantasy]",Classic,unknown


In [34]:

ratings

,username,anime_id,title,genre,type,my_score,my_watched_episodes,aired_from_year,time_period
0,--FallenAngel--,164,Mononoke Hime,"[Action, Adventure, Fantasy]",Movie,10,1,1997,Classic
1,--FallenAngel--,5507,Senjou no Valkyria,"[Action, Military, Romance]",TV,10,26,2009,Classic
2,--FallenAngel--,17265,Log Horizon,"[Action, Game, Adventure, Magic, Fantasy]",TV,10,25,2013,New-Gen
3,--FallenAngel--,6547,Angel Beats!,"[Action, Comedy, Drama, School, Supernatural]",TV,10,13,2010,New-Gen
4,--FallenAngel--,10800,Chihayafuru,"[Drama, Game, Josei, School, Slice of Life...",TV,10,25,2011,New-Gen
...,...,...,...,...,...,...,...,...,...
92895,zzs,60,Chrno Crusade,"[Action, Historical, Demons, Supernatural, ...",TV,8,24,2003,Classic
92896,zzs,1974,Glass no Kamen (2005),"[Drama, Shoujo]",TV,8,51,2005,Classic
92897,zzs,269,Bleach,"[Action, Adventure, Comedy, Super Power, S...",TV,8,207,2004,Classic
92898,zzs,4898,Kuroshitsuji,"[Action, Comedy, Demons, Fantasy, Historic...",TV,8,24,2008,Classic
